In [24]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [25]:
from scipy.io import loadmat
import numpy as np
mnist=loadmat(r"C:\Users\archa\OneDrive\Desktop\Downloads\mnist-original.mat")
mnist

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Sun Mar 30 03:19:02 2014',
 '__version__': '1.0',
 '__globals__': [],
 'mldata_descr_ordering': array([[array(['label'], dtype='<U5'), array(['data'], dtype='<U4')]],
       dtype=object),
 'data': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'label': array([[0., 0., 0., ..., 9., 9., 9.]])}

In [26]:
data=mnist['data']
data.shape

(784, 70000)

In [27]:
label=mnist['label']
label.shape

(1, 70000)

In [28]:
x=mnist['data'].T
x.shape

(70000, 784)

In [29]:
y=mnist['label'][0]
y.shape

(70000,)

In [30]:
y

array([0., 0., 0., ..., 9., 9., 9.])

In [31]:
xtrain=x[np.any([y==1,y==2],axis=0)]
ytrain=y[np.any([y==1,y==2],axis=0)]

In [32]:
print(xtrain)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [33]:
xtrain.shape

(14867, 784)

In [34]:
print(ytrain)
ytrain.shape

[1. 1. 1. ... 2. 2. 2.]


(14867,)

In [35]:
#normalising the data set
xtrain=xtrain/255.0
xtrain=xtrain.T
print(xtrain.shape)

(784, 14867)


In [36]:
ndim=xtrain.shape[0]
ndim

784

In [37]:
ytrain=ytrain.reshape(1,ytrain.shape[0])
ytrain.shape

(1, 14867)

In [38]:
ytrain=ytrain-1
ytrain

array([[0., 0., 0., ..., 1., 1., 1.]])

In [41]:
tf.reset_default_graph()
x=tf.placeholder(tf.float32,[ndim,None])
y=tf.placeholder(tf.float32,[1,None])
learning_rate=tf.placeholder(tf.float32,shape=())
w=tf.Variable(tf.zeros([1,ndim]))
b=tf.Variable(tf.zeros(1))
init=tf.global_variables_initializer()
y_=tf.sigmoid(tf.matmul(w,x)+b)
cost=-tf.reduce_mean(y*tf.log(y_)+(1-y)*tf.log(1-y_))
training_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [42]:
def run_logistic_model(learning_r,training_epocs,train_obs,train_labels,debug=False):
    sess=tf.Session()
    sess.run(init)
    cost_history=np.empty(shape=[0],dtype=float)
    for epoch in range(training_epocs+1):
        sess.run(training_step,feed_dict={x:train_obs,y:train_labels,learning_rate:learning_r})
        cost_=sess.run(cost,feed_dict={x:train_obs,y:train_labels})
        cost_history=np.append(cost_history,cost_)
        if (epoch % 500==0) & debug:
            print('reached epoch',epoch,' cost j',cost_)
    return sess,cost_history

In [43]:
sess,cost_history=run_logistic_model(learning_r=0.01,training_epocs=1500,train_obs=xtrain,train_labels=ytrain,debug=True)

reached epoch 0  cost j 0.6785402
reached epoch 500  cost j 0.10848413
reached epoch 1000  cost j 0.07872986
reached epoch 1500  cost j 0.06656062


In [44]:
correct_prediction=tf.equal(tf.greater(y_,0.5),tf.equal(y,1))
print(correct_prediction)

Tensor("Equal_1:0", shape=(1, ?), dtype=bool)


In [46]:
print(sess.run(correct_prediction,feed_dict={x:xtrain,y:ytrain}))

[[ True  True  True ...  True  True  True]]


In [48]:
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print(sess.run(accuracy,feed_dict={x:xtrain,y:ytrain}))

0.9854712


In [49]:
threshold=0.5

In [50]:
predictions=tf.cast(tf.greater_equal(y_,threshold),tf.int32)

In [56]:
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [57]:
print(sess.run(accuracy,feed_dict={x:xtrain,y:ytrain}))

0.9854712
